In [ ]:
import cv2
from glob import glob
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

In [ ]:
# image_dir='Z:/ortofotomapa/images/'
image_dir='/mnt/nfs/ortofotomapa/images'
tiffs = glob(image_dir+'/*.tif')
len(tiffs)

In [ ]:
fname = tiffs[0]
print(fname)

In [ ]:
img = cv2.imread(fname)
img.shape

In [ ]:
plt.imshow(img[:,:,::-1])

# Rasterio

In [ ]:
import rasterio
from rasterio.plot import show

In [ ]:
raster = rasterio.open(fname)
show(raster)

In [ ]:
dir(raster)

In [ ]:
print(raster.width, raster.height)
print(raster.crs)
print(raster.transform)
print(raster.count)
print(raster.indexes)
print(raster.bounds)


In [ ]:
print(raster.transform)

In [ ]:
raster.transform * (0,0), raster.transform * (raster.width, raster.height)

In [ ]:
plt.imshow(raster.read().transpose(1,2,0))
raster.read().transpose(1,2,0).shape

In [ ]:
ymax=800000
ymin=150000
xmin=150000
xmax=800000
scale=1/5
themap=np.zeros((int((ymax-ymin)*scale), int((xmax-xmin)*scale), 3),np.uint8)
print(themap.shape, themap.dtype)

In [ ]:
def plot_on_map(themap, raster, xmin,ymin,scale):
    (ixmin,iymax),(ixmax,iymin) = raster.transform * (0,0), raster.transform * (raster.width, raster.height)
    ixmin-=xmin
    ixmax-=xmin
    iymin-=ymin
    iymax-=ymin
    
    ixmin*=scale
    iymin*=scale
    ixmax*=scale
    iymax*=scale
    
    ixmin = int(ixmin)
    iymin = int(iymin)
    ixmax = int(ixmax)
    iymax = int(iymax)
    
    npr = raster.read().transpose(1,2,0)
    npr = cv2.resize(npr, (ixmax-ixmin, iymax-iymin))
    themap[iymin:iymax, ixmin:ixmax] = npr[::-1]


for fn in tqdm(tiffs):
    raster = rasterio.open(fn)
    plot_on_map(themap,raster,xmin,ymin,scale)
plt.figure(figsize=(40,40))
plt.imshow(themap)
cv2.imwrite('/tmp/map.png', themap)
print('DONE')

In [ ]:
plt.figure(figsize=(40,40))
plt.imshow(themap[::-1,:,::-1])
cv2.imwrite('/tmp/map.png', themap)


In [ ]:
fn

In [ ]:
len(tiffs)

# EOMaps?

In [ ]:
# from eomaps import Maps


# m = Maps(crs=2180, figsize=(10, 10))
# m.add_wms.OpenStreetMap.add_layer.stamen_terrain()
# # plot shapefile (zorder=2 to be on top of the DEM)
# # m.add_gdf(shp_data, column=shp_data.VEGKOD, cmap="viridis", ec="k", lw=0.2, alpha=0.5, zorder=2)
# # plot DEM
# m2 = m.new_layer_from_file.GeoTIFF(tiffs[0], zorder=1)

# m.ax.set_extent((589913.0408156103, 713614.6619114348, 7495264.310799116, 7618965.93189494),
#                 Maps.CRS.epsg(2180))

In [ ]:
import geopandas as gpd

In [ ]:

shp_data = gpd.read_file("/mnt/nfs/ortofotomapa/shp/unzipped/SHP/PL.PZGiK.333.0801__OT_BUBD_A.shp")
shp_data

In [ ]:
plt.figure(figsize=(20,10))
shp_data.X_KOD.hist()

In [ ]:
shp_data.X_KOD.value_counts()

In [ ]:
shp_data.geometry.plot()

In [ ]:
shp_data.crs

In [ ]:
shp_data.geometry.iloc[0]

In [ ]:
shp_data.geometry.iloc[0].exterior

In [ ]:
shp_data.geometry.iloc[0].exterior.xy

# Arkusze

In [ ]:
# godlo1992 = gpd.read_file(r"C:\Users\karol\projects\cufix\bdot10kseg\data\SiatkaPodzialuArkuszowegoNMT_PL1992_5000\PL1992_5000.shp")
godlo1992 = gpd.read_file('/mnt/nfs/ortofotomapa/shp/SiatkaPodzialuArkuszowegoNMT_PL1992_5000/PL1992_5000.shp')

godlo1992

In [ ]:
godlo1992.geometry.plot(figsize=(10,10))

In [ ]:
downloaded_godlos = []
for t in tiffs:
    downloaded_godlos.append(t.split('/')[-1].split('_')[-1][:-4])

In [ ]:
downloaded_godlos[:15]

In [ ]:
'M-34-9-B-c-4-1' in godlo1992.godlo

In [ ]:
godlo1992['downloaded'] = [_ in downloaded_godlos for _ in godlo1992.godlo]
godlo1992

In [ ]:
godlo1992.downloaded.value_counts()

In [ ]:
godlo1992[godlo1992['downloaded']==True].geometry.plot(figsize=(20,20))


In [ ]:
godlo1992[godlo1992['downloaded']==False].geometry.plot(figsize=(20,20))

In [ ]:
godlo1992[godlo1992['downloaded']==True].geometry